# Sistemas Operacionais

##Laboratório 2 - João Marcos Silva Ferandes de Freitas

#**Exercício 1**

Modifique o código acima para fazer com que dois processos filhos alterem uma memória compartilhada contendo um inteiro. Faça o processo pai esperar por seus dois filhos para depois imprimir o valor da variável na tela. Dica: olhe o código do exercício 3 do Laboratório 1.



In [1]:
%%writefile exe1.c
#include <sys/types.h>
#include <sys/wait.h>
#include <stdio.h>
#include <unistd.h>
#include <sys/shm.h>
#include <sys/stat.h>
#include <stdlib.h>

int main(int argc, char **argv) {

    int segment_id = shmget(IPC_PRIVATE, sizeof(int), S_IRUSR | S_IWUSR);

    int *shared_memory = (int *)shmat(segment_id, NULL, 0);
    *shared_memory = 0;

    for (int i = 0; i < 2; i++) {
        pid_t pid = fork();

        if (pid < 0) {
            printf("Erro ao criar processo filho\n");
            return 1;
        } else if (pid == 0) {
            *shared_memory += 100;
            printf("Filho %d alterou o valor para: %d\n", getpid(), *shared_memory);

            shmdt(shared_memory);
            exit(0);
        }
    }

    wait(NULL);
    wait(NULL);

    printf("Valor final na memória compartilhada: %d\n", *shared_memory);


    shmdt(shared_memory);
    shmctl(segment_id, IPC_RMID, NULL);

    return 0;
}

Writing exe1.c


In [2]:
!gcc ./exe1.c -o exe1
!./exe1

Filho 699 alterou o valor para: 100
Filho 698 alterou o valor para: 200
Valor final na memória compartilhada: 200


#**Exercício 2**

Utilizando comunicação entre processos, crie um código que escreva a Sequência de Fibonacci, com o parâmetro n sendo uma variável global. Tal sequência deve ser gerada por um processo filho, mas deve ser impressa na tela pelo processo pai. Dica: olhe o código do desafio do Laboratório 1.


In [4]:
%%writefile exe2.c
#include <sys/types.h>
#include <sys/wait.h>
#include <stdio.h>
#include <unistd.h>
#include <sys/shm.h>
#include <sys/stat.h>
#include <stdlib.h>

int n = 20;

int main(int argc, char **argv) {
    int segment_id = shmget(IPC_PRIVATE, (n+1)*sizeof(int), S_IRUSR | S_IWUSR);
    int *fib = (int *)shmat(segment_id, NULL, 0);

    pid_t pid = fork();

    if (pid < 0) {
        printf("Erro ao criar processo filho\n");
        return 1;
    } else if (pid == 0) {
        fib[0] = 0;
        if (n > 0) fib[1] = 1;

        for (int i = 2; i <= n; i++) {
            fib[i] = fib[i-1] + fib[i-2];
        }

        shmdt(fib);
        exit(0);
    } else { // Processo pai
        wait(NULL);

        printf("Sequência de Fibonacci (primeiros %d termos):\n", n);
        for (int i = 0; i <= n; i++) {
            printf("%d ", fib[i]);
        }
        printf("\n");

        // Limpeza
        shmdt(fib);
        shmctl(segment_id, IPC_RMID, NULL);
    }

    return 0;
}

Writing exe2.c


In [5]:
!gcc ./exe2.c -o exe2
!./exe2

Sequência de Fibonacci (primeiros 20 termos):
0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 


#**Exercício 3**

Usando comunicação entre processos, escreva um código que realiza a soma de *n* números naturais, sendo *n* uma variável global. Cada processo deve fazer apenas a soma de dois números e o resultado final deve ser retornado pelo primeiro processo. Por exemplo, o processo 1 deve fazer a soma de 0+1, o processo 2 vai pegar esse resultado e somar com 2, e assim por diante.


In [6]:
%%writefile exe3.c
#include <sys/types.h>
#include <sys/wait.h>
#include <stdio.h>
#include <unistd.h>
#include <sys/shm.h>
#include <sys/stat.h>
#include <stdlib.h>

int n = 10;

int main(int argc, char **argv) {

    int segment_id = shmget(IPC_PRIVATE, sizeof(int), S_IRUSR | S_IWUSR);
    int *result = (int *)shmat(segment_id, NULL, 0);
    *result = 0;

    for (int i = 0; i <= n; i++) {
        pid_t pid = fork();

        if (pid < 0) {
            printf("Erro ao criar processo\n");
            return 1;
        } else if (pid == 0) {
            *result += i;
            printf("Processo %d: somou %d, resultado parcial = %d\n", getpid(), i, *result);

            shmdt(result);
            exit(0);
        } else {
            wait(NULL);
        }
    }

    printf("Soma dos primeiros %d números naturais: %d\n", n, *result);

    shmdt(result);
    shmctl(segment_id, IPC_RMID, NULL);

    return 0;
}

Writing exe3.c


In [7]:
!gcc ./exe3.c -o exe3
!./exe3

Processo 1224: somou 0, resultado parcial = 0
Processo 1225: somou 1, resultado parcial = 1
Processo 1226: somou 2, resultado parcial = 3
Processo 1227: somou 3, resultado parcial = 6
Processo 1228: somou 4, resultado parcial = 10
Processo 1229: somou 5, resultado parcial = 15
Processo 1230: somou 6, resultado parcial = 21
Processo 1231: somou 7, resultado parcial = 28
Processo 1232: somou 8, resultado parcial = 36
Processo 1233: somou 9, resultado parcial = 45
Processo 1234: somou 10, resultado parcial = 55
Soma dos primeiros 10 números naturais: 55
